In [1]:
from z3 import *

# Q1: BMC model checking of $G p$ and $F p$

In [40]:
class s:
    '''
    s(n) is the state s_n.
    variables is supposed to be global 
    '''
    def __init__(self,n):
        self.vars=[Bool(var+str(n)) for var in variables]
    def get_vars(self):
        return self.vars
    

In [3]:
def G(p):
    '''
    checks the LTL formula (A) G p
    p is a function that takes state and gives a z3 expression saying whether property holds in that state
    '''
    states=[s(0)]                                                #variable states stores s_0,s_1...s_k
    for k in range(1,maxk+1):
        solver=Solver()
        states+=[s(k)]
        initial=I(states[0])                                      #s_0 is initial state
        Trans=And([T(states[i],states[i+1]) for i in range(k)])   #/\ T(s_i,s_i+1)
        Notp=Or([Not(p(state)) for state in states])              #\/ ~p(s_i)
        solver.add(And(initial,Trans,Notp))
        if solver.check()==sat:
            return f'Cex found of length {k+1}. Property not True'
    return f'No Cex found till length {maxk}'
        

def F(p):
    states=[s(0),s(1)]                                            #variable states stores s_0,s_1...s_k
    for k in range(2,maxk+1):
        solver=Solver()
        states+=[s(k)]
        initial=I(states[0])                                      #s_0 is initial state
        Trans=And([T(states[i],states[i+1]) for i in range(k)])   #/\ T(s_i,s_i+1)
        Notp=And([Not(p(state)) for state in states])             #/\ ~p(s_i)
        loop=Or([Equals(states[i],states[-1]) for i in range(k)]) #\/ s_l=s_k+1
        solver.add(And(initial,Trans,Notp,loop))
        if solver.check()==sat:
            return f'Cex lasso path of length {k} found. Property not True'
    return f'No Cex found till length {maxk}'
    
    

In [4]:
def I(s):
    '''
    Checks whether s is initial state. Gives the z3 encoding of this constraint
    
    '''
    v=s.get_vars()
    constraints=[And([v[i]==intial[i] for i in range(len(v))]) for intial in init]
    return Or(*constraints)

def T(s1,s2):
    '''
    Gives the z3 encoding of tranistion relation.
    
    '''
    return Trans(*(s1.get_vars()),*(s2.get_vars()))
    

def Equals(s1,s2):
    '''
    checks whether s1 and s2 are same. Returns z3 encoding of the constraint
    '''
    return And([s==sp for s,sp in zip(s1.get_vars(),s2.get_vars())])



# Initializing

The process for initializing is very simple
- Store all variables in an array named `variables`
- Store the initial states as an array in he array `init`
- Set the maximum k in `maxk`
- Define the transition relation `Trans(s,s')` where `s` refers to state variables and `s'` refers to the state variables after transition


In [ ]:
variables=['start','close','heat','error']                               #Boolean variables of states
init=[[False,False,False,False]]                                         #array of initial states
maxk=6
def Trans(s,c,h,e,sp,cp,hp,ep):
    '''
    custom Transition relation. This one is for microwave.
    
    '''
    return Or(And(Not(s),Not(c),Not(h),Not(e),Not(sp),cp,Not(hp),Not(ep)),
              And(Not(s),Not(c),Not(h),Not(e),sp,Not(cp),Not(hp),ep),
              And(Not(s),c,Not(h),Not(e),Not(sp),Not(cp),Not(hp),Not(ep)),
              And(Not(s),c,Not(h),Not(e),sp,cp,Not(hp),Not(ep)),
              And(s,Not(c),Not(h),e,sp,cp,Not(hp),ep),
              And(s,c,Not(h),e,sp,Not(cp),Not(hp),ep),
              And(s,c,Not(h),e,Not(sp),cp,Not(hp),Not(ep)),
              And(Not(s),c,h,Not(e),Not(sp),cp,hp,Not(ep)),
              And(Not(s),c,h,Not(e),Not(sp),cp,Not(hp),Not(ep)),
              And(Not(s),c,h,Not(e),Not(sp),Not(cp),Not(hp),Not(ep)),
              And(s,c,Not(h),Not(e),sp,cp,hp,Not(ep)),
              And(s,c,h,Not(e),Not(sp),cp,hp,Not(ep)))

# Verifying properties

This model checker only supports LTL formulae $G p$ and $F p$
- define a function `p(s)` which returns the z3 encoded formula of the property p
- To check say $G p$, run `G(p)`

### checking property $G(start \implies (close \lor error))$

In [6]:
def p(s):
    start,close,heat,error=s.get_vars()
    return Implies(start,Or(close,error))

G(p)

'No Cex found till length 6'

### checking property $G(\lnot close \lor \lnot error)$

In [22]:
def p(s):
    start,close,heat,error=s.get_vars()
    return Or(Not(close),Not(error))

G(p)

'Cex found of length 3. Property not True'

### checking property $F(error)$

In [8]:
def p(s):
    start,close,heat,error=s.get_vars()
    return error

F(p)

'Cex lasso path of length 2 found. Property not True'

### checking property $F (close)$

In [9]:
def p(s):
    start,close,heat,error=s.get_vars()
    return close

F(p)

'No Cex found till length 6'

# checking property $F((\lnot start \implies close) \land (start \implies heat))$

In [10]:
def p(s):
    start,close,heat,error=s.get_vars()
    return And(Implies(Not(start),close),Implies(start,heat))

F(p)

'Cex lasso path of length 3 found. Property not True'

# Recurrance diameter

Calculates the recurrance diameter of the Kripke structure formed by transition relation given above

In [11]:
def rec_diam():
    k=2
    states=[s(0),s(1)]
    while True:
        solver=Solver()
        states+=[s(k)]
        initial=I(states[0])
        Trans=And([T(states[i],states[i+1]) for i in range(k)])
        loop=Not(Or([Equals(states[i],states[j]) for i in range(k+1) for j in range(i+1,k+1)]))
        solver.add(And(initial,Trans,loop))
        if solver.check()==unsat:
            return f'recurrance diameter is {k-1}'
        k+=1
    
        

In [12]:
rec_diam()

'recurrance diameter is 6'

# k induction

Complete k induction procedure to check property $G p$.
- Just as before, define `p(s)`
- `k_induct(p)` will verify the property $G p$

In [36]:
def k_induct(p):
    states=[s(0)]
    init_solver=Solver()
    initial=I(states[0])
    init_solver.add(And(initial,Not(p(states[0]))))
    if init_solver.check()==sat:
        return 'Cex found. Property not true'
    k=1
    while True:
        states+=[s(k)]
        Notps=Or([Not(p(state)) for state in states])
        Ps=And([p(states[i]) for i in range(k)])
        Trans=And([T(states[i],states[i+1]) for i in range(k)])
        Notp=Not(p(states[-1]))
        notequal=Not(Or([Equals(states[i],states[j]) for i in range(k+1) for j in range(i+1,k+1)]))
        base_solver=Solver()
        base_solver.add(Notps,initial,Trans)
        if base_solver.check()==sat:
            return 'Cex found. Property not true'
        induct_solver=Solver()
        induct_solver.add(And(Ps,Trans,Notp,notequal))
        if induct_solver.check()==unsat:
            return 'Property is True'
        k=k+1
        
        

In [37]:
def p(s):
    start,close,heat,error=s.get_vars()
    return Implies(start,Or(close,error))

k_induct(p)

'Property is True'

In [38]:
def p(s):
    start,close,heat,error=s.get_vars()
    return Or(Not(close),Not(error))

k_induct(p)

'Cex found. Property not true'

In [39]:
def p(s):
    start,close,heat,error=s.get_vars()
    return Implies(And(close,start),Or(Not(heat),Not(error)))

k_induct(p)

'Property is True'